<a href="https://colab.research.google.com/github/nihal-rao/deepchem/blob/master/baselines/FREESOLV_GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installing DeepChem

In [1]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3501  100  3501    0     0  25554      0 --:--:-- --:--:-- --:--:-- 25554


add /root/miniconda/lib/python3.7/site-packages to PYTHONPATH
python version: 3.7.10
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added conda-forge to channels
added omnia to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [2]:
!pip install --pre deepchem
!pip install dgl
!pip install dgllife

     |████████████████████████████████| 552kB 4.4MB/s 


We can now import the `deepchem` package to play with.

In [3]:
import deepchem as dc
dc.__version__

'2.6.0.dev'

## Using GraphConv featuriser

Implementing and recording the baseline for Tox21 dataset.

In [4]:
featurizer = dc.feat.MolGraphConvFeaturizer()
tasks, datasets, transformers = dc.molnet.load_sampl(featurizer=featurizer)
train_dataset, valid_dataset, test_dataset = datasets
print(train_dataset)

Failed to featurize datapoint 61, N. Appending empty array
Exception message: zero-size array to reduction operation maximum which has no identity
Failed to featurize datapoint 195, S. Appending empty array
Exception message: zero-size array to reduction operation maximum which has no identity
Failed to featurize datapoint 286, C. Appending empty array
Exception message: zero-size array to reduction operation maximum which has no identity
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


<DiskDataset X.shape: (511,), y.shape: (511, 1), w.shape: (511, 1), ids: ['CS(=O)(=O)Cl' 'CC(C)C=C' 'CCCCCCCO' ... 'CC1=CCCCC1'
 'CC1=CC[C@H](C[C@@H]1O)C(=C)C' 'CC(=C)[C@H]1CCC(=CC1)C=O'], task_names: ['expt']>


ECFP featurizer is used. Extended Connectivity Fingerprints  is a **fingerprinting** method. They are also sometimes called "circular fingerprints". The ECFP algorithm begins by classifying atoms based only on their direct properties and bonds. Each unique pattern is a feature. For example, "carbon atom bonded to two hydrogens and two heavy atoms" would be a feature, and a particular element of the fingerprint is set to 1 for any molecule that contains that feature. It then iteratively identifies new features by looking at larger circular neighborhoods. One specific feature bonded to two other specific features becomes a higher level feature, and the corresponding element is set for any molecule that contains it. This continues for a fixed number of iterations, most often two.

In [ ]:
tasks

['expt']

Above are the tasks in the Tox21 dataset - there are 12 tasks, each corresponding to different biotoxicity targets, such as cell receptors and stress response pathways.

In [5]:
datasets

(<DiskDataset X.shape: (511,), y.shape: (511, 1), w.shape: (511, 1), ids: ['CS(=O)(=O)Cl' 'CC(C)C=C' 'CCCCCCCO' ... 'CC1=CCCCC1'
  'CC1=CC[C@H](C[C@@H]1O)C(=C)C' 'CC(=C)[C@H]1CCC(=CC1)C=O'], task_names: ['expt']>,
 <DiskDataset X.shape: (64,), y.shape: (64, 1), w.shape: (64, 1), ids: ['c1ccc2cc(ccc2c1)O' 'Cc1ccc2cc(ccc2c1)C' 'c1ccc2ccccc2c1' ...
  'c1cnc[nH]1' 'Cn1ccnc1' 'Cc1c[nH]cn1'], task_names: ['expt']>,
 <DiskDataset X.shape: (64,), y.shape: (64, 1), w.shape: (64, 1), ids: ['CN1CCNCC1' 'CN1CCN(CC1)C' 'C1CNCCN1' ... 'Cc1cccc(c1C)Nc2ccccc2C(=O)O'
  'C1CCCC(CC1)O' 'c1ccc2c(c1)CCC2'], task_names: ['expt']>)

There are three dataset objects - train split, val split and test split. Each split consists of X and y - X is the features and y is the output label. Froe example the train split has X.shape (513, 1024)
and y.shape (513, 1). This implies that there are 513 samples in the train split - and each sample is represented by an ECFP vector of size 1024. 

##Training the GCNNN

In [11]:
from deepchem.models import GCNModel
model = GCNModel(mode='regression', n_tasks=len(tasks), batch_size=32, learning_rate=0.001)
model.fit(train_dataset, nb_epoch=15)

0.07689919471740722

`MultitaskClassifier` is a simple stack of fully connected layers. A single hidden layer of width 1000 is used. Each input will have 1024 features, and it should produce predictions for 12 different tasks.

Note that the above network is performing multitask learning - a single network is used for all 12 tasks. This is because inter task correlations exist in the data, and to take if advantage of this single neural network is used for multiple tasks.

In [12]:
import numpy as np

metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print('training set score:', model.evaluate(train_dataset, [metric], transformers))
print('test set score:', model.evaluate(test_dataset, [metric], transformers))

training set score: {'pearson_r2_score': 0.9636048341814806}
test set score: {'pearson_r2_score': 0.6673799269906617}


The training set score is much higher than test set score. This indicates overfitting - and is why metrics on the validation set need to be measured in otder to tune parameters and detect overfitting.